In [21]:
# For secrets
import sys
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()

best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']
result = rpc_connection.getrawtransaction('0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098',1)
#print(result)
test = rpc_connection.getblockhash(113288)
tests = rpc_connection.getblock(test)['tx']
for k in tests:
    print(k)
    result = rpc_connection.getrawtransaction(k,1)
    print(result)




110fac349d9d37297f9a0327437a608f5509e87b0bb74926f3e0defc9bc3f42a
{'txid': '110fac349d9d37297f9a0327437a608f5509e87b0bb74926f3e0defc9bc3f42a', 'hash': '110fac349d9d37297f9a0327437a608f5509e87b0bb74926f3e0defc9bc3f42a', 'version': 1, 'size': 134, 'vsize': 134, 'weight': 536, 'locktime': 0, 'vin': [{'coinbase': '0431dc001b0103', 'sequence': 4294967295}], 'vout': [{'value': Decimal('50.02000000'), 'n': 0, 'scriptPubKey': {'asm': '049f86bb7f7d94ec5208d48013fb15afb167123971c4ee0c7e3e78dab0c88a23431ec00f193c66d7b4704f945fc2ceddbd3b0d666642263a863ddcac00468b55b2 OP_CHECKSIG', 'hex': '41049f86bb7f7d94ec5208d48013fb15afb167123971c4ee0c7e3e78dab0c88a23431ec00f193c66d7b4704f945fc2ceddbd3b0d666642263a863ddcac00468b55b2ac', 'reqSigs': 1, 'type': 'pubkey', 'addresses': ['1NPFfriqH61cA2Zjk9GbZ3DsWu35ycT5jr']}}], 'hex': '01000000010000000000000000000000000000000000000000000000000000000000000000ffffffff070431dc001b0103ffffffff018076242a010000004341049f86bb7f7d94ec5208d48013fb15afb167123971c4ee0c7e3e78da

In [22]:
def is_mi_cond(in_addrs, out_addrs):
    #if in_addrs[0] == 'coinbase':
    if len(in_addrs) < 2:
        return False
    if len(out_addrs) > 2:
        return False
    return True

def all_same_cls_num(addrs, addr_set):
    cls_num_set = set()
    for addr in addr_set.keys():
        cls_num_set.add(addr_set[addr])
    if len(cls_num_set) == 1:
        return True
    else:
        return False
        

def get_min_cluster(addr_set, flag=0):
    '''flag: 0 전체 최소값
       flag: 1 -1이 아닌 최소값'''
    cluster_num_list = list()
    for addr in addr_set.keys():
        cluster_num_list.append(addr_set[addr])
    sort_cls_num_list = sorted(cluster_num_list)
    if flag == 0:
        return sort_cls_num_list[0]
    elif flag == 1:
        for num in sort_cls_num_list:
            if num > -1:
                return num
    
def get_cluster_num(addrs, max_cluster_num, addr_set):
    cls_num = -1
    if len(addr_set.keys()) == 0:
        return -1
    if all_same_cls_num(addrs, addr_set):
        cls_num = int(get_min_cluster(addr_set, 0))
        if cls_num == -1:
            cls_num = int(max_cluster_num) + 1
            max_cluster_num = cls_num
    else:
        cls_num = get_min_cluster(addr_set, 1)
    return cls_num, max_cluster_num

In [23]:
#DB Query
import sqlite3

db_index = '/home/dnlab/BitcoinBlockSampler/index.db'
db_txhash = '/media/dnlab/0602da39-763c-42b0-b186-f929ac6b3f66/200529/txhash.db'
index_conn = sqlite3.connect(db_index)
txhash_conn = sqlite3.connect(db_txhash)
icur = index_conn.cursor()
tcur = txhash_conn.cursor()


def get_txid(txhash):
    try:
        icur.execute('''SELECT DISTINCT id FROM TxID WHERE txhash = '{}'; '''.format(txhash))
        tx_indexes = icur.fetchall()
        return tx_indexes[0][0]

    except Exception as e:
        return None


def get_addr_txin(tx_indexes):
    try:
        tcur.execute('''SELECT DISTINCT addr FROM TxIn WHERE tx = '{}'; '''.format(tx_indexes))
        address_list = [addr[0] for addr in tcur.fetchall()]
        return set(address_list)

    except Exception as e:
        return None


def get_addr_txout(tx_indexes):
    try:
        tcur.execute('''SELECT DISTINCT addr FROM TxOut WHERE tx = '{}'; '''.format(tx_indexes))
        address_list = [addr[0] for addr in tcur.fetchall()]
        return set(address_list)
    except Exception as e:
        return None



In [24]:
db_path = '/home/dnlab/BitcoinBlockSampler/cluster.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()


def create_meta_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Meta (
                     key TEXT PRIMARY KEY,
                     value TEXT);
                ''')
    
    
def update_meta_table(key, value):
    cur.execute('''INSERT OR IGNORE INTO Meta (
                        key, value) VALUES (
                        ?, ?);
                ''', (key, value))
    cur.execute('''UPDATE Meta SET value = ? WHERE key = ?;
                ''', (value, key))
    

def get_meta(key):
    cur.execute('''SELECT value FROM Meta WHERE key = ?''', (key,))
    result = cur.fetchone()
    if result is not None:
        result = result[0]
    return result
    
    
def create_cluster_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Cluster (
                     address INTEGER PRIMARY KEY,
                     number INTEGER NOT NULL);
                ''')
    
    
def insert_cluster(address, number):
    cur.execute('''INSERT OR IGNORE INTO Cluster (
                       address, number) VALUES (
                       ?, ?);
                    ''', (address, number))

    
def insert_cluster_many(addr_list):
    cur.execute('''INSERT OR IGNORE INTO Cluster VALUES(?, ?)''',addr_list)
    
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

    
def commit_transactions():
    cur.execute('COMMIT;')

    
def get_min_cluster(addresses):
    cur.execute(f'''SELECT MIN(number) FROM Cluster WHERE address IN ('{",".join(addrss)}')''')
    return cur.fetchone()[0]


def get_cluster_number(address):
    cur.execute(f'''SELECT number FROM Cluster WHERE address IN ('{",".join(addrss)}')''')
    cls_num = set([addr_tuple[0] for addr_tuple in cur.fetchall()])
    return cls_num


def get_all_cluster():
    try:
        cur.execute('''SELECT DISTINCT * FROM Cluster; ''')
        addr_dict = dict()
        for addr in cur.fetchall():
            addr_dict.update({addr[0]:addr[1]})
        return addr_dict
    except Exception as e:
        return None

In [ ]:
# SingleProcess
import multiprocessing
import sys
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import time
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()

pool_num = multiprocessing.cpu_count()//2
term = 0
best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']

term = 1000
start_height = 0
end_height = 400000

#begin_transactions()
update_meta_table('max_num', -1)
conn.commit()

cluster_num_set = set()
starttime = 0
endtime = 0    
starttime = time.time()
addr_set = dict()


for i in range(0, longest_height):
    max_cluster_num = int(get_meta('max_num'))
    if i % 10000 == 0:
        print("BLOCK HEIGHT: ", i, "TIME: ", endtime-starttime)
        
    block_hash = rpc_connection.getblockhash(i)
    txes = rpc_connection.getblock(block_hash)['tx']
    for tx in txes:
        tx_indexes = get_txid(tx)
        in_addrs = get_addr_txin(tx_indexes)
        out_addrs = get_addr_txout(tx_indexes)
        
        if is_mi_cond(in_addrs, out_addrs):
            cluster_num, max_cluster_num = get_cluster_num(in_addrs, max_cluster_num, addr_set)
        else:
            cluster_num = -1
        #print("ADDR INFO: in_addr:{}, out_addr:{}".format(len(in_addrs), len(out_addrs)))
        
        for in_addr in in_addrs:
            addr_set[in_addr] = addr_set.get(in_addr, cluster_num)
        for out_addr in out_addrs:
            addr_set[out_addr] = addr_set.get(out_addr, -1)
        #print("ADDRSET", addr_set)
        
    begin_transactions()
    update_meta_table('max_num', max_cluster_num)
    commit_transactions()
    begin_transactions()
    if len(addr_set) == 0:
        continue
    for addr in addr_set.keys():
        insert_cluster(addr, addr_set[addr])
    commit_transactions()
    endtime = time.time()
    #print("COMMIT COMPLETE;")

In [25]:
def multi_input(height):
    print(height)
    addr_set = get_all_cluster()
    max_cluster_num = get_meta('max_num')
    if addr_set == None:
        return {}, max_cluster_num
    block_hash = rpc_connection.getblockhash(height)
    try:
        txes = rpc_connection.getblock(block_hash)['tx']
    except Exception as e:
        print(e)
        return addr_set, max_cluster_num

    for tx in txes:
        #print("TX",tx)
        tx_indexes = get_txid(tx)
        in_addrs = get_addr_txin(tx_indexes)
        out_addrs = get_addr_txout(tx_indexes)
        if tx_indexes == None or in_addrs == None or out_addrs == None:
            return addr_set, max_cluster_num
        if is_mi_cond(in_addrs, out_addrs):
            cluster_num, max_cluster_num = get_cluster_num(in_addrs, max_cluster_num, addr_set)
        else:
            cluster_num = -1
        #print("HEIGT: ", height, "ADDR INFO: in_addr:{}, out_addr:{}".format(len(in_addrs), len(out_addrs)))
        for in_addr in in_addrs:
            addr_set[in_addr] = addr_set.get(in_addr, cluster_num)
        for out_addr in out_addrs:
            addr_set[out_addr] = addr_set.get(out_addr, -1)
    return addr_set, max_cluster_num
        #print("ADDRSET", addr_set)

In [26]:
create_meta_table()
create_cluster_table()

In [20]:
# multiprocess
import multiprocessing
import sys
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import time
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()

pool_num = multiprocessing.cpu_count()//2
term = 0
best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']

term = 1000
start_height = 0
end_height = 100000

update_meta_table('max_num', -1)
conn.commit()

cluster_num_set = set()
starttime = 0
endtime = 0    
starttime = time.time()
addr_set = dict()

for sheight, eheight in zip(range(start_height, end_height,term), range(start_height + term, end_height + term, term)):
    stime = time.time()
    if eheight >= end_height:
        eheight = end_height + 1
    begin_transactions()
    with multiprocessing.Pool(pool_num) as p:
        results = p.imap(multi_input, range(sheight, eheight))
        for addrset, max_num in results:
            #print("[FINALRESULT]",addrset, max_num)
            update_meta_table('max_num', max_num)
            for k, v in addrset.items():
                insert_cluster(k, v)
    commit_transactions()
    etime = time.time()
    print(f'TxIn Job done from {sheight} to {eheight-1} during {etime-stime}')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
END
END
END
END
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
END
END
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
END
END
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
END
END
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
END
END
85
86
87
88
89
90
91
92
93
94
95
96
97
-1: JSON value is not a string as expected
string indices must be integers
98
99
100
101
102
103
END
END
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
END
END
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
END
END
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
END
END
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
END
END
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
END
END
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
END
END
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232

JSONRPCException: -342: non-JSON HTTP response with '400 Bad Request' from server

In [ ]:
def multi_input(TX_list):
    count = 0
    addr_dict = dict()
    TX_set = set(TX_list)
    test = TX_set.pop()
    cluster_num = 0
    while len(TX_set) > 0:
        tx = TX_set.pop()
        if is_mi_cond(tx[0]):
            addrs = addr_from_txin(tx[0])
            cluster_num = get_cluster_num(addrs)
            #print("[DEBUG] update cluster:", addrs, cluster_num)
            update_cluster(addrs, cluster_num)
        count += 1
        print("[DONE] {}%".format(str(count/len(tx_list) * 100)))
    return addr_dict
cur.execute(f'''SELECT * FROM TxHash''')
tx_list = cur.fetchall()
print(len(tx_list))
multi_input(tx_list)

In [ ]:
import sqlite3
db_path = '/media/dnlab/0602da39-763c-42b0-b186-f929ac6b3f66/200529/txhash.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()
def get_max_height():
    cur.execute('''SELECT MAX(height) FROM TxHash;''')
    return cur.fetchone()[0]

def get_cluster_number(addresses):
    cur.execute(f'''SELECT MIN(number) FROM Cluster WHERE address IN ({",".join(addresses)})''')
    return cur.fetchone()[0]

In [ ]:
get_max_height()
addrss = {'1Q1UVrJJB3nsZ8b4jeCQKrp9S2RboYudcP'}
print(f'''SELECT * FROM TxHash''')
cur.execute(f'''SELECT * FROM TxHash''')
len(cur.fetchall())

In [ ]:
rpc_connection = get_rpc()
result = rpc_connection.getrawtransaction('44bafb2099e68eac125c579a68de0360474dbf85b81c0f45ab760955ef432f39',1)
print(result)

In [ ]:
{'txid': '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098', 
 'hash': '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098', 
 'version': 1, 'size': 134, 'vsize': 134, 'weight': 536, 'locktime': 0, 
 'vin': [{'coinbase': '04ffff001d0104', 'sequence': 4294967295}], 
 'vout': [{'value': Decimal('50.00000000'), 'n': 0, 
           'scriptPubKey': {'asm': '0496b538e853519c726a2c91e61ec11600ae1390813a627c66fb8be7947be63c52da7589379515d4e0a604f8141781e62294721166bf621e73a82cbf2342c858ee OP_CHECKSIG', 
                            'hex': '410496b538e853519c726a2c91e61ec11600ae1390813a627c66fb8be7947be63c52da7589379515d4e0a604f8141781e62294721166bf621e73a82cbf2342c858eeac', 
                            'reqSigs': 1, 
                            'type': 'pubkey', 
                            'addresses': ['12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX']}}], 
 'hex': '01000000010000000000000000000000000000000000000000000000000000000000000000ffffffff0704ffff001d0104ffffffff0100f2052a0100000043410496b538e853519c726a2c91e61ec11600ae1390813a627c66fb8be7947be63c52da7589379515d4e0a604f8141781e62294721166bf621e73a82cbf2342c858eeac00000000', 
 'blockhash': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 
 'confirmations': 632697, 
 'time': 1231469665, 
 'blocktime': 1231469665}

In [27]:
# SingleProcess
import multiprocessing
import sys
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import time
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()

pool_num = multiprocessing.cpu_count()//2
term = 0
best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']

term = 1000
start_height = 0
end_height = 400000

#begin_transactions()
update_meta_table('max_num', -1)
conn.commit()

cluster_num_set = set()
starttime = 0
endtime = 0    
starttime = time.time()
addr_set = dict()
for i in range(0, longest_height):
    max_cluster_num = int(get_meta('max_num'))
    if i % 10000 == 0:
        print("BLOCK HEIGHT: ", i, "TIME: ", endtime-starttime)
    block_hash = rpc_connection.getblockhash(i)
    txes = rpc_connection.getblock(block_hash)['tx']
    for tx in txes:
        tx_indexes = get_txid(tx)
        in_addrs = get_addr_txin(tx_indexes)
        out_addrs = get_addr_txout(tx_indexes)
        if is_mi_cond(in_addrs, out_addrs):
            cluster_num, max_cluster_num = get_cluster_num(in_addrs, max_cluster_num, addr_set)
        else:
            cluster_num = -1
        #print("ADDR INFO: in_addr:{}, out_addr:{}".format(len(in_addrs), len(out_addrs)))
        for in_addr in in_addrs:
            addr_set[in_addr] = addr_set.get(in_addr, cluster_num)
        for out_addr in out_addrs:
            addr_set[out_addr] = addr_set.get(out_addr, -1)
        #print("ADDRSET", addr_set)
    begin_transactions()
    update_meta_table('max_num', max_cluster_num)
    commit_transactions()
    begin_transactions()
    if len(addr_set) == 0:
        continue
    for addr in addr_set.keys():
        insert_cluster(addr, addr_set[addr])
    commit_transactions()
    endtime = time.time()
    #print("COMMIT COMPLETE;")

BLOCK HEIGHT:  0 TIME:  -1591605528.1732213


KeyboardInterrupt: 

In [ ]:
commit_transactions()

In [ ]:
# multiprocess
import multiprocessing
import sys
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import time
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()

pool_num = multiprocessing.cpu_count()//2
term = 0
best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']

term = 1000
start_height = 0
end_height = 400000

update_meta_table('max_num', -1)

cluster_num_set = set()
starttime = 0
endtime = 0    
starttime = time.time()

for i in range(0, longest_height):
    addr_set = get_all_cluster()
    max_cluster_num = int(get_meta('max_num'))
    if i % 10000 == 0:
        print("BLOCK HEIGHT: ", i, "TIME: ", endtime-starttime)
    block_hash = rpc_connection.getblockhash(i)
    txes = rpc_connection.getblock(block_hash)['tx']
    for tx in txes:
        tx_indexes = get_txid(tx)
        in_addrs = get_addr_txin(tx_indexes)
        out_addrs = get_addr_txout(tx_indexes)
        if is_mi_cond(in_addrs, out_addrs):
            cluster_num, max_cluster_num = get_cluster_num(in_addrs, max_cluster_num, addr_set)
        else:
            cluster_num = -1
        for in_addr in in_addrs:
            addr_set[in_addr] = addr_set.get(in_addr, cluster_num)
        for out_addr in out_addrs:
            addr_set[out_addr] = addr_set.get(out_addr, -1)
    update_meta_table('max_num', max_num)
    if len(addrset) == 0:
        continue
    for addr in addrset.keys():
        insert_cluster(addr, addrset[addr])
    endtime = time.time()


In [ ]:
def multi_input(txes):
    addr_set = dict()
    for tx in txes:
        tx_indexes = get_txid(tx)
        in_addrs = get_addr_txin(tx_indexes)
        out_addrs = get_addr_txout(tx_indexes)
        
        if is_mi_cond(in_addrs, out_addrs):
            cluster_num, max_cluster_num = get_cluster_num(in_addrs, max_cluster_num, addr_set)
        else:
            cluster_num = -1
            
        for in_addr in in_addrs:
            addr_set[in_addr] = addr_set.get(in_addr, cluster_num)
        for out_addr in out_addrs:
            addr_set[out_addr] = addr_set.get(out_addr, -1)
    return addr_set, max_cluster_num
    
def get_min_cluster(addr_set, flag=0):
    cluster_num_list = list()
    for addr in addr_set.keys():
        cluster_num_list.append(addr_set[addr])
    sort_cls_num_list = sorted(cluster_num_list)
    if flag == 0:
        return sort_cls_num_list[0]
    elif flag == 1:
        for num in sort_cls_num_list:
            if num > -1:
                return num
            
def is_mi_cond(in_addrs, out_addrs):
    if len(in_addrs) < 2:
        return False
    if len(out_addrs) > 2:
        return False
    return True

In [40]:
%time y = [-1]*1000000000

CPU times: user 2.41 s, sys: 1.97 s, total: 4.38 s
Wall time: 4.37 s
